<h1 id = "PP"> 
<div style = "padding:20px;color:#ffffff;margin:0;font-size:150%;text-align:center;display:fill;border-radius:5px;background-color:#0974ba;overflow:hidden;font-weight:800"> 
    IMPORTANT INFO!   
</div>
</h1>  

<div style = "color:white; display:fill; border-radius:5px; background-color:#093145; font-size:90%; font-family:Verdana; letter-spacing:0.5px">
<p style = "padding:10px; color:white; margin-bottom:0;">
Last year's notebook was way to large and it took forever to load as a result. I decided to split up the notebook in 3 parts - EDA, Matchups & Stats, and the Machine Learning Model. The links to these notebooks are found below. The notebooks are a work in progress as the 2025 tournament teams have not been selected yet. 

<div class = "alert alert-block alert-info" style = "font-size:90%; font-family:Verdana; line-height: 1.7em; letter-spacing:0.5px">
<b> Notebook Links </b> <ul>
<li> EDA - https://www.kaggle.com/code/nishaanamin/bracketology-part-1-eda
<li> Matchups & Stats - https://www.kaggle.com/code/nishaanamin/bracketology-part-2-matchups-stats
<li> Machine Learning Model - https://www.kaggle.com/code/nishaanamin/bracketology-part-3-machine-learning-model  
</div>

<h1 id = "PP">
<div style = "padding:20px;color:#ffffff;margin:0;font-size:150%;text-align:center;display:fill;border-radius:5px;background-color:#0974ba;overflow:hidden;font-weight:800">
    Preprocessing      
</div>
</h1>  

In [1]:
# Importing Libraries  
from functools import reduce
import gc  
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe 
from IPython.display import display, HTML  
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go 
import plotly.io as pio  
import plotly.offline as py   
from plotly.subplots import make_subplots 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import MinMaxScaler  
import seaborn as sns 
import warnings 
from xgboost import XGBRegressor  

pio.renderers.default = 'iframe' 
pd.set_option('display.max_columns', None)  
warnings.filterwarnings('ignore')  

HTML("""
<style>
g.pointtext {display: none;}
</style>
""")

In [2]:
# Read All Datasets 
fte_df = pd.read_csv('/kaggle/input/march-madness-data/538 Ratings.csv')
ban_df = pd.read_csv('/kaggle/input/march-madness-data/Barttorvik Away-Neutral.csv')
ba_df = pd.read_csv('/kaggle/input/march-madness-data/Barttorvik Away.csv')
bh_df = pd.read_csv('/kaggle/input/march-madness-data/Barttorvik Home.csv')
bn_df = pd.read_csv('/kaggle/input/march-madness-data/Barttorvik Neutral.csv')
coach_res_df = pd.read_csv('/kaggle/input/march-madness-data/Coach Results.csv')
conf_res_df = pd.read_csv('/kaggle/input/march-madness-data/Conference Results.csv')
cs_df = pd.read_csv('/kaggle/input/march-madness-data/Conference Stats.csv')
csh_df = pd.read_csv('/kaggle/input/march-madness-data/Conference Stats Home.csv')
csa_df = pd.read_csv('/kaggle/input/march-madness-data/Conference Stats Away.csv')
csn_df = pd.read_csv('/kaggle/input/march-madness-data/Conference Stats Neutral.csv')
csan_df = pd.read_csv('/kaggle/input/march-madness-data/Conference Stats Away Neutral.csv')
em_df = pd.read_csv('/kaggle/input/march-madness-data/EvanMiya.csv')  
hcti_df = pd.read_csv('/kaggle/input/march-madness-data/Heat Check Tournament Index.csv')
kb_df = pd.read_csv('/kaggle/input/march-madness-data/KenPom Barttorvik.csv')  
pv_df = pd.read_csv('/kaggle/input/march-madness-data/Preseason Votes.csv')
pp_df = pd.read_csv('/kaggle/input/march-madness-data/Public Picks.csv')
r_df = pd.read_csv('/kaggle/input/march-madness-data/Resumes.csv')
sr_df = pd.read_csv('/kaggle/input/march-madness-data/Seed Results.csv')
ss_df = pd.read_csv('/kaggle/input/march-madness-data/Shooting Splits.csv')
tr_df = pd.read_csv('/kaggle/input/march-madness-data/Team Results.csv')
tl_df = pd.read_csv('/kaggle/input/march-madness-data/Tournament Locations.csv')
tm_df = pd.read_csv('/kaggle/input/march-madness-data/Tournament Matchups.csv')  
ts_df = pd.read_csv('/kaggle/input/march-madness-data/Tournament Simulation.csv') 
uc_df = pd.read_csv('/kaggle/input/march-madness-data/Upset Count.csv')
usi_df = pd.read_csv('/kaggle/input/march-madness-data/Upset Seed Info.csv')

In [3]:
# Global Variables 
curr_year = 2024 
prev_year = curr_year - 1   
tournament_count = 15   

order_seed = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]    

round_str_64 = ['CHAMPION', 'FINALS', 'FINAL 4', 'ELITE 8', 'SWEET 16', 'SECOND ROUND', 'FIRST ROUND'] 
round_str_68 = ['CHAMPION', 'FINALS', 'FINAL 4', 'ELITE 8', 'SWEET 16', 'SECOND ROUND', 'FIRST ROUND', 'FIRST FOUR']  

# round_str_reverse_64 = ['FIRST ROUND', 'SECOND ROUND', 'SWEET 16', 'ELITE 8', 'FINAL 4', 'FINALS', 'CHAMPION']  
# round_str_reverse_68 = ['FIRST FOUR', 'FIRST ROUND', 'SECOND ROUND', 'SWEET 16', 'ELITE 8', 'FINAL 4', 'FINALS', 'CHAMPION'] 

rounds_num_64 = [1, 2, 4, 8, 16, 32, 64] 
rounds_num_68 = [1, 2, 4, 8, 16, 32, 64, 68]

In [4]:
# Combine Datasets to create one comprehensive dataset for all tournament teams 
############################################################################### 

temp_kb_df = kb_df.copy()

temp_bh_df = bh_df.copy()  
# Add Prefix to indicate type of team stats (H = Home, A = Away, N = Neutral, AN = Away Neutral)
temp_bh_df = temp_bh_df.add_prefix('H ')                               
# Rename H TEAM NO column to TEAM NO because it will be the column used to merge 
temp_bh_df = temp_bh_df.rename(columns = {'H TEAM NO' : 'TEAM NO'}) 

temp_ba_df = ba_df.copy()
temp_ba_df = temp_ba_df.add_prefix('A ')
temp_ba_df = temp_ba_df.rename(columns = {'A TEAM NO' : 'TEAM NO'}) 

temp_bn_df = bn_df.copy()
temp_bn_df = temp_bn_df.add_prefix('N ')
temp_bn_df = temp_bn_df.rename(columns = {'N TEAM NO' : 'TEAM NO'}) 

temp_ban_df = ban_df.copy()
temp_ban_df = temp_ban_df.add_prefix('AN ')
temp_ban_df = temp_ban_df.rename(columns = {'AN TEAM NO' : 'TEAM NO'}) 

temp_hcti_df = hcti_df.copy()
temp_hcti_df = temp_hcti_df.rename(columns = {'YEAR' : 'HCTI YEAR', 'TEAM' : 'HCTI TEAM', 'SEED' : 'HCTI SEED', 'ROUND' : 'HCTI ROUND'}) 

temp_pv_df = pv_df.copy()
temp_pv_df = temp_pv_df.rename(columns = {'YEAR' : 'PV YEAR', 'TEAM' : 'PV TEAM', 'SEED' : 'PV SEED', 'ROUND' : 'PV ROUND'}) 

temp_ss_df = ss_df.copy()
temp_ss_df = temp_ss_df.rename(columns = {'YEAR' : 'SS YEAR', 'TEAM ID' : 'SS TEAM ID', 'TEAM' : 'SS TEAM', 'CONF' : 'SS CONF'}) 

temp_r_df = r_df.copy()
temp_r_df = temp_r_df.rename(columns = {'YEAR' : 'R YEAR', 'SEED' : 'R SEED', 'TEAM' : 'R TEAM', 'ROUND' : 'R ROUND'}) 

temp_em_df = em_df.copy()
temp_em_df = temp_em_df.rename(columns = {'YEAR' : 'EM YEAR', 'SEED' : 'EM SEED', 'TEAM' : 'EM TEAM', 'ROUND' : 'EM ROUND'}) 
           
dfs = [temp_kb_df, temp_bh_df, temp_ba_df, temp_bn_df, temp_ban_df, temp_hcti_df, temp_pv_df, temp_ss_df, temp_r_df, temp_em_df]

# Merge all columns to make one dataframe consisting of all team stats 
complete_stats = reduce(lambda left, right: pd.merge(left, right, on = ['TEAM NO'], how = 'left'), dfs)  
# Drop redundant columns 
complete_stats.drop(['H YEAR', 'H TEAM ID', 'H TEAM', 'H SEED', 'H ROUND', 'A YEAR', 'A TEAM ID', 'A TEAM', 'A SEED', 'A ROUND', 
                     'N YEAR', 'N TEAM ID', 'N TEAM', 'N SEED', 'N ROUND', 'AN YEAR', 'AN TEAM ID', 'AN TEAM', 'AN SEED', 'AN ROUND',
                     'GAMES', 'W', 'L', 
                     'H GAMES', 'H W', 'H L', 'H TALENT', 'H EXP', 'H AVG HGT', 'H EFF HGT', 'A GAMES', 'A W', 'A L', 'A TALENT', 'A EXP', 'A AVG HGT', 'A EFF HGT', 
                     'N GAMES', 'N W', 'N L', 'N TALENT', 'N EXP', 'N AVG HGT', 'N EFF HGT', 'AN GAMES', 'AN W', 'AN L', 'AN TALENT', 'AN EXP', 'AN AVG HGT', 'AN EFF HGT',
                     'OP OREB%', 'OP DREB%', 'H OP OREB%', 'H OP DREB%', 'A OP OREB%', 'A OP DREB%', 'N OP OREB%', 'N OP DREB%', 'AN OP OREB%', 'AN OP DREB%',
                     'THREES FG%', 'THREES SHARE', 'THREES FG%D', 'THREES D SHARE', 
                     'H TALENT RANK', 'H EXP RANK', 'H AVG HGT RANK', 'H EFF HGT RANK', 'A TALENT RANK', 'A EXP RANK', 'A AVG HGT RANK', 'A EFF HGT RANK', 
                     'N TALENT RANK', 'N EXP RANK', 'N AVG HGT RANK', 'N EFF HGT RANK', 'AN TALENT RANK', 'AN EXP RANK', 'AN AVG HGT RANK', 'AN EFF HGT RANK',
                     'OP OREB% RANK', 'OP DREB% RANK', 
                     'H OP OREB% RANK', 'H OP DREB% RANK', 'A OP OREB% RANK', 'A OP DREB% RANK', 'N OP OREB% RANK', 'N OP DREB% RANK', 'AN OP OREB% RANK', 'AN OP DREB% RANK',
                     'THREES FG% RANK', 'THREES SHARE RANK', 'THREES FG%D RANK', 'THREES D SHARE RANK', 
                     'HCTI YEAR', 'HCTI TEAM', 'HCTI SEED', 'HCTI ROUND', 'PV YEAR', 'PV TEAM', 'PV SEED', 'PV ROUND', 
                     'SS YEAR', 'SS TEAM ID', 'SS TEAM', 'SS CONF',
                     'R YEAR', 'R SEED', 'R TEAM', 'R ROUND', 'WINS', 'RANK?', 
                     'EM YEAR', 'EM SEED', 'EM TEAM', 'EM ROUND',], 
                     inplace = True, axis = 1)

# Create new stats  
complete_stats['A BADJ EM RANK DIFF'] = complete_stats['BADJ EM RANK'] - complete_stats['A BADJ EM RANK']
complete_stats['AN BADJ EM RANK DIFF'] = complete_stats['BADJ EM RANK'] - complete_stats['AN BADJ EM RANK']
complete_stats['A BADJ O RANK DIFF'] = complete_stats['BADJ O RANK'] - complete_stats['A BADJ O RANK']
complete_stats['AN BADJ O RANK DIFF'] = complete_stats['BADJ O RANK'] - complete_stats['AN BADJ O RANK']
complete_stats['A BADJ D RANK DIFF'] = complete_stats['BADJ D RANK'] - complete_stats['A BADJ D RANK']
complete_stats['AN BADJ D RANK DIFF'] = complete_stats['BADJ D RANK'] - complete_stats['AN BADJ D RANK'] 

# Remove the First Four Round as that is irrelevant to the EDA and Machine Learning Model  
complete_stats = complete_stats.loc[complete_stats['ROUND'].ne(68)]
complete_stats.head()  

,YEAR,CONF,CONF ID,QUAD NO,QUAD ID,TEAM NO,TEAM ID,TEAM,SEED,ROUND,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FARTHER TWOS FG% RANK,FARTHER TWOS SHARE RANK,F

In [5]:
# Create dataframe of tournament matchups  
#########################################  

temp_complete_stats = complete_stats.drop(['YEAR', 'TEAM', 'SEED', 'ROUND'], axis = 1)
# Create temp dataframe too add sections of it to the tm dataframe 
temp_tournament_matchups = pd.merge(tm_df, temp_complete_stats, how = 'left', on = ['TEAM NO']) 
tournament_matchups = pd.DataFrame()

temp_tournament_matchups['OUTCOME'] = ''
rounds = list(reversed(rounds_num_64))[: - 1] 

# Create Outcome column to indicate winning and losing teams per matchup (0 = Losing Team, 1 = Winning Team)
for rnd in rounds : 
    df = temp_tournament_matchups.loc[temp_tournament_matchups['CURRENT ROUND'] == rnd]
    df = df.reset_index(drop = True) 
    df.loc[df['ROUND'] != rnd, 'OUTCOME'] = 1 
    df.loc[df['ROUND'] == rnd, 'OUTCOME'] = 0
    tournament_matchups = pd.concat([tournament_matchups, df], ignore_index = True, sort = False)
    
tournament_matchups.head()  

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,CONF,CONF ID,QUAD NO,QUAD ID,TEAM ID,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FART

In [6]:
# Order the teams in W - L order  
################################  

# Create dataframe of all winning teams 
win = tournament_matchups.loc[tournament_matchups['OUTCOME'] == 1] 
win = win.reset_index(drop = True)   

# Create dataframe of all losing teams 
loss = tournament_matchups.loc[tournament_matchups['OUTCOME'] == 0] 
loss = loss.reset_index(drop = True)   

# Change indexes of rows to have the winning team as the first row and the losing team as the second row per two rows 
win['INDEX'] = np.arange(0, len(win) * 2, 2)
loss['INDEX'] = np.arange(1, len(win) * 2, 2)

# Set the index column as the row index 
win = win.set_index('INDEX')
loss = loss.set_index('INDEX')

# Combine the win and loss dataframes to create the tournament matchups in order 
ordered_tournament_matchups = pd.concat([win, loss], axis = 0)
ordered_tournament_matchups = ordered_tournament_matchups.rename_axis(None, axis = 0)
ordered_tournament_matchups  = ordered_tournament_matchups.sort_index(ascending = True)
ordered_tournament_matchups2 = ordered_tournament_matchups.copy()

ordered_tournament_matchups.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,CONF,CONF ID,QUAD NO,QUAD ID,TEAM ID,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FART

In [7]:
# Scale variables between a value of 0 to 1 for the tournament matchups  
scaler = MinMaxScaler() 
removed_cols = ['YEAR', 'BY YEAR NO', 'BY ROUND NO', 'TEAM NO', 'TEAM', 'SEED', 'ROUND', 'CURRENT ROUND', 'CONF', 'CONF ID', 'QUAD NO', 'QUAD ID', 'TEAM ID', 'BID TYPE', 'OUTCOME']   
selected_cols = ordered_tournament_matchups.columns[~ordered_tournament_matchups.columns.isin(removed_cols)]
ordered_tournament_matchups[selected_cols] = scaler.fit_transform(ordered_tournament_matchups[selected_cols])  
ordered_tournament_matchups.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,CONF,CONF ID,QUAD NO,QUAD ID,TEAM ID,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FART

In [8]:
# Scale variables between a value of 0 to 1 for statistics of all teams  
complete_stats_scaled = complete_stats.copy()
selected_cols = complete_stats_scaled.columns[~complete_stats_scaled.columns.isin(removed_cols)] 
complete_stats_scaled[selected_cols] = scaler.fit_transform(complete_stats_scaled[selected_cols])  
complete_stats_scaled.head()

,YEAR,CONF,CONF ID,QUAD NO,QUAD ID,TEAM NO,TEAM ID,TEAM,SEED,ROUND,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FARTHER TWOS FG% RANK,FARTHER TWOS SHARE RANK,F

In [9]:
# Put the winning and losing teams in one row  
#############################################

odds = ordered_tournament_matchups.copy()
# Get all rows of odd index 
odds_df = odds.iloc[1::2]
# Change the columns to have the prefix "L" which indicates the losing teams 
odds_df = odds_df.add_prefix('L ')
odds_df = odds_df.reset_index(drop = True)

evens = ordered_tournament_matchups.copy()
# Get all rows of evens index  
evens_df = evens.iloc[::2]
# Change the columns to have the prefix "W" which indicates the winning teams  
evens_df = evens_df.add_prefix('W ')
evens_df = evens_df.reset_index(drop = True)

# Combine the odds and evens dataframes to create the combined tournament matchups  
tournament_matchups_combined_rows = pd.concat([evens_df, odds_df], axis = 1) 
tournament_matchups_combined_rows = tournament_matchups_combined_rows.sort_index(ascending = True)
tournament_matchups_combined_rows.head()

,W YEAR,W BY YEAR NO,W BY ROUND NO,W TEAM NO,W TEAM,W SEED,W ROUND,W CURRENT ROUND,W SCORE,W CONF,W CONF ID,W QUAD NO,W QUAD ID,W TEAM ID,W K TEMPO,W K TEMPO RANK,W KADJ T,W KADJ T RANK,W K OFF,W KO RANK,W KADJ O,W KADJ O RANK,W K DEF,W KD RANK,W KADJ D,W KADJ D RANK,W KADJ EM,W KADJ EM RANK,W BADJ EM,W BADJ O,W BADJ D,W BARTHAG,W WIN%,W EFG%,W EFG%D,W FTR,W FTRD,W TOV%,W TOV%D,W OREB%,W DREB%,W RAW T,W 2PT%,W 2PT%D,W 3PT%,W 3PT%D,W BLK%,W BLKED%,W AST%,W OP AST%,W 2PTR,W 3PTR,W 2PTRD,W 3PTRD,W BADJ T,W AVG HGT,W EFF HGT,W EXP,W TALENT,W FT%,W OP FT%,W PPPO,W PPPD,W ELITE SOS,W WAB,W BADJ EM RANK,W BADJ O RANK,W BADJ D RANK,W BARTHAG RANK,W EFG% RANK,W EFGD% RANK,W FTR RANK,W FTRD RANK,W TOV% RANK,W TOV%D RANK,W OREB% RANK,W DREB% RANK,W RAW T RANK,W 2PT% RANK,W 2PT%D RANK,W 3PT% RANK,W 3PT%D RANK,W BLK% RANK,W BLKED% RANK,W AST% RANK,W OP AST% RANK,W 2PTR RANK,W 3PTR RANK,W 2PTRD RANK,W 3PTRD RANK,W BADJT RANK,W AVG HGT RANK,W EFF HGT RANK,W EXP RANK,W TALENT RANK,W FT% RANK,W OP FT% RANK,W PPPO RANK,W PPPD RANK,W ELITE SOS RANK,W H BADJ EM,W H BADJ O,W H BADJ D,W H BARTHAG,W H WIN%,W H EFG%,W H EFG%D,W H FTR,W H FTRD,W H TOV%,W H TOV%D,W H OREB%,W H DREB%,W H RAW T,W H 2PT%,W H 2PT%D,W H 3PT%,W H 3PT%D,W H BLK%,W H BLKED%,W H AST%,W H OP AST%,W H 2PTR,W H 3PTR,W H 2PTRD,W H 3PTRD,W H BADJ T,W H FT%,W H OP FT%,W H PPPO,W H PPPD,W H ELITE SOS,W H WAB,W H BADJ EM RANK,W H BADJ O RANK,W H BADJ D RANK,W H BARTHAG RANK,W H EFG% RANK,W H EFGD% RANK,W H FTR RANK,W H FTRD RANK,W H TOV% RANK,W H TOV%D RANK,W H OREB% RANK,W H DREB% RANK,W H RAW T RANK,W H 2PT% RANK,W H 2PT%D RANK,W H 3PT% RANK,W H 3PT%D RANK,W H BLK% RANK,W H BLKED% RANK,W H AST% RANK,W H OP AST% RANK,W H 2PTR RANK,W H 3PTR RANK,W H 2PTRD RANK,W H 3PTRD RANK,W H BADJT RANK,W H FT% RANK,W H OP FT% RANK,W H PPPO RANK,W H PPPD RANK,W H ELITE SOS RANK,W A BADJ EM,W A BADJ O,W A BADJ D,W A BARTHAG,W A WIN%,W A EFG%,W A EFG%D,W A FTR,W A FTRD,W A TOV%,W A TOV%D,W A OREB%,W A DREB%,W A RAW T,W A 2PT%,W A 2PT%D,W A 3PT%,W A 3PT%D,W A BLK%,W A BLKED%,W A AST%,W A OP AST%,W A 2PTR,W A 3PTR,W A 2PTRD,W A 3PTRD,W A BADJ T,W A FT%,W A OP FT%,W A PPPO,W A PPPD,W A ELITE SOS,W A WAB,W A BADJ EM RANK,W A BADJ O RANK,W A BADJ D RANK,W A BARTHAG RANK,W A EFG% RANK,W A EFGD% RANK,W A FTR RANK,W A FTRD RANK,W A TOV% RANK,W A TOV%D RANK,W A OREB% RANK,W A DREB% RANK,W A RAW T RANK,W A 2PT% RANK,W A 2PT%D RANK,W A 3PT% RANK,W A 3PT%D RANK,W A BLK% RANK,W A BLKED% RANK,W A AST% RANK,W A OP AST% RANK,W A 2PTR RANK,W A 3PTR RANK,W A 2PTRD RANK,W A 3PTRD RANK,W A BADJT RANK,W A FT% RANK,W A OP FT% RANK,W A PPPO RANK,W A PPPD RANK,W A ELITE SOS RANK,W N BADJ EM,W N BADJ O,W N BADJ D,W N BARTHAG,W N WIN%,W N EFG%,W N EFG%D,W N FTR,W N FTRD,W N TOV%,W N TOV%D,W N OREB%,W N DREB%,W N RAW T,W N 2PT%,W N 2PT%D,W N 3PT%,W N 3PT%D,W N BLK%,W N BLKED%,W N AST%,W N OP AST%,W N 2PTR,W N 3PTR,W N 2PTRD,W N 3PTRD,W N BADJ T,W N FT%,W N OP FT%,W N PPPO,W N PPPD,W N ELITE SOS,W N WAB,W N BADJ EM RANK,W N BADJ O RANK,W N BADJ D RANK,W N BARTHAG RANK,W N EFG% RANK,W N EFGD% RANK,W N FTR RANK,W N FTRD RANK,W N TOV% RANK,W N TOV%D RANK,W N OREB% RANK,W N DREB% RANK,W N RAW T RANK,W N 2PT% RANK,W N 2PT%D RANK,W N 3PT% RANK,W N 3PT%D RANK,W N BLK% RANK,W N BLKED% RANK,W N AST% RANK,W N OP AST% RANK,W N 2PTR RANK,W N 3PTR RANK,W N 2PTRD RANK,W N 3PTRD RANK,W N BADJT RANK,W N FT% RANK,W N OP FT% RANK,W N PPPO RANK,W N PPPD RANK,W N ELITE SOS RANK,W AN BADJ EM,W AN BADJ O,W AN BADJ D,W AN BARTHAG,W AN WIN%,W AN EFG%,W AN EFG%D,W AN FTR,W AN FTRD,W AN TOV%,W AN TOV%D,W AN OREB%,W AN DREB%,W AN RAW T,W AN 2PT%,W AN 2PT%D,W AN 3PT%,W AN 3PT%D,W AN BLK%,W AN BLKED%,W AN AST%,W AN OP AST%,W AN 2PTR,W AN 3PTR,W AN 2PTRD,W AN 3PTRD,W AN BADJ T,W AN FT%,W AN OP FT%,W AN PPPO,W AN PPPD,W AN ELITE SOS,W AN WAB,W AN BADJ EM RANK,W AN BADJ O RANK,W AN BADJ D RANK,W AN BARTHAG RANK,W AN EFG% RANK,W AN EFGD% RANK,W AN FTR RANK,W AN FTRD RANK,W AN TOV% RANK,W AN TOV%D RANK,W AN OREB% RANK,W AN DREB% RANK,W AN RAW T RANK,W AN 2PT% RA

In [10]:
# Get difference of variables between winning and losing team 
#############################################################

removed_cols = ['YEAR', 'BY YEAR NO', 'BY ROUND NO', 'TEAM NO', 'TEAM', 'SEED', 'ROUND', 'CURRENT ROUND', 'CONF', 'CONF ID', 'QUAD NO', 'QUAD ID', 'TEAM ID', 'BID TYPE', 'OUTCOME'] 
selected_cols = tournament_matchups.columns[~tournament_matchups.columns.isin(removed_cols)] 

# Get the difference of every 2 rows 
odds = ordered_tournament_matchups[selected_cols].diff()
# Select all rows of odd index  
odds_df = odds.iloc[1::2]

# Get the difference of every 2 rows and flip the sign 
evens = - ordered_tournament_matchups[selected_cols].diff()
# Shift the values up one row  
evens = evens[selected_cols].shift(- 1)
# Select all rows of even index   
evens_df = evens.iloc[::2]

temp = ordered_tournament_matchups[removed_cols]

# Create dataframe of all matchup differentials 
ordered_differentials = pd.concat([temp, evens_df], axis = 1) 
ordered_differentials = pd.concat([ordered_differentials, odds_df], axis = 0) 
ordered_differentials = ordered_differentials.groupby(level = 0).sum()
ordered_differentials = ordered_differentials.sort_index(ascending = True)

ordered_differentials.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,CONF,CONF ID,QUAD NO,QUAD ID,TEAM ID,BID TYPE,OUTCOME,SCORE,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS 

In [11]:
# Put the winning and losing teams in one row   
#############################################

win = ordered_differentials.loc[ordered_differentials['OUTCOME'] == 1] 
win = win.add_prefix('W ')
win = win.reset_index(drop = True)   

loss = ordered_differentials.loc[ordered_differentials['OUTCOME'] == 0] 
loss = loss.add_prefix('L ')
loss = loss.reset_index(drop = True)  

differentials_combined_rows = pd.concat([win, loss], axis = 1)
differentials_combined_rows.head()  

,W YEAR,W BY YEAR NO,W BY ROUND NO,W TEAM NO,W TEAM,W SEED,W ROUND,W CURRENT ROUND,W CONF,W CONF ID,W QUAD NO,W QUAD ID,W TEAM ID,W BID TYPE,W OUTCOME,W SCORE,W K TEMPO,W K TEMPO RANK,W KADJ T,W KADJ T RANK,W K OFF,W KO RANK,W KADJ O,W KADJ O RANK,W K DEF,W KD RANK,W KADJ D,W KADJ D RANK,W KADJ EM,W KADJ EM RANK,W BADJ EM,W BADJ O,W BADJ D,W BARTHAG,W WIN%,W EFG%,W EFG%D,W FTR,W FTRD,W TOV%,W TOV%D,W OREB%,W DREB%,W RAW T,W 2PT%,W 2PT%D,W 3PT%,W 3PT%D,W BLK%,W BLKED%,W AST%,W OP AST%,W 2PTR,W 3PTR,W 2PTRD,W 3PTRD,W BADJ T,W AVG HGT,W EFF HGT,W EXP,W TALENT,W FT%,W OP FT%,W PPPO,W PPPD,W ELITE SOS,W WAB,W BADJ EM RANK,W BADJ O RANK,W BADJ D RANK,W BARTHAG RANK,W EFG% RANK,W EFGD% RANK,W FTR RANK,W FTRD RANK,W TOV% RANK,W TOV%D RANK,W OREB% RANK,W DREB% RANK,W RAW T RANK,W 2PT% RANK,W 2PT%D RANK,W 3PT% RANK,W 3PT%D RANK,W BLK% RANK,W BLKED% RANK,W AST% RANK,W OP AST% RANK,W 2PTR RANK,W 3PTR RANK,W 2PTRD RANK,W 3PTRD RANK,W BADJT RANK,W AVG HGT RANK,W EFF HGT RANK,W EXP RANK,W TALENT RANK,W FT% RANK,W OP FT% RANK,W PPPO RANK,W PPPD RANK,W ELITE SOS RANK,W H BADJ EM,W H BADJ O,W H BADJ D,W H BARTHAG,W H WIN%,W H EFG%,W H EFG%D,W H FTR,W H FTRD,W H TOV%,W H TOV%D,W H OREB%,W H DREB%,W H RAW T,W H 2PT%,W H 2PT%D,W H 3PT%,W H 3PT%D,W H BLK%,W H BLKED%,W H AST%,W H OP AST%,W H 2PTR,W H 3PTR,W H 2PTRD,W H 3PTRD,W H BADJ T,W H FT%,W H OP FT%,W H PPPO,W H PPPD,W H ELITE SOS,W H WAB,W H BADJ EM RANK,W H BADJ O RANK,W H BADJ D RANK,W H BARTHAG RANK,W H EFG% RANK,W H EFGD% RANK,W H FTR RANK,W H FTRD RANK,W H TOV% RANK,W H TOV%D RANK,W H OREB% RANK,W H DREB% RANK,W H RAW T RANK,W H 2PT% RANK,W H 2PT%D RANK,W H 3PT% RANK,W H 3PT%D RANK,W H BLK% RANK,W H BLKED% RANK,W H AST% RANK,W H OP AST% RANK,W H 2PTR RANK,W H 3PTR RANK,W H 2PTRD RANK,W H 3PTRD RANK,W H BADJT RANK,W H FT% RANK,W H OP FT% RANK,W H PPPO RANK,W H PPPD RANK,W H ELITE SOS RANK,W A BADJ EM,W A BADJ O,W A BADJ D,W A BARTHAG,W A WIN%,W A EFG%,W A EFG%D,W A FTR,W A FTRD,W A TOV%,W A TOV%D,W A OREB%,W A DREB%,W A RAW T,W A 2PT%,W A 2PT%D,W A 3PT%,W A 3PT%D,W A BLK%,W A BLKED%,W A AST%,W A OP AST%,W A 2PTR,W A 3PTR,W A 2PTRD,W A 3PTRD,W A BADJ T,W A FT%,W A OP FT%,W A PPPO,W A PPPD,W A ELITE SOS,W A WAB,W A BADJ EM RANK,W A BADJ O RANK,W A BADJ D RANK,W A BARTHAG RANK,W A EFG% RANK,W A EFGD% RANK,W A FTR RANK,W A FTRD RANK,W A TOV% RANK,W A TOV%D RANK,W A OREB% RANK,W A DREB% RANK,W A RAW T RANK,W A 2PT% RANK,W A 2PT%D RANK,W A 3PT% RANK,W A 3PT%D RANK,W A BLK% RANK,W A BLKED% RANK,W A AST% RANK,W A OP AST% RANK,W A 2PTR RANK,W A 3PTR RANK,W A 2PTRD RANK,W A 3PTRD RANK,W A BADJT RANK,W A FT% RANK,W A OP FT% RANK,W A PPPO RANK,W A PPPD RANK,W A ELITE SOS RANK,W N BADJ EM,W N BADJ O,W N BADJ D,W N BARTHAG,W N WIN%,W N EFG%,W N EFG%D,W N FTR,W N FTRD,W N TOV%,W N TOV%D,W N OREB%,W N DREB%,W N RAW T,W N 2PT%,W N 2PT%D,W N 3PT%,W N 3PT%D,W N BLK%,W N BLKED%,W N AST%,W N OP AST%,W N 2PTR,W N 3PTR,W N 2PTRD,W N 3PTRD,W N BADJ T,W N FT%,W N OP FT%,W N PPPO,W N PPPD,W N ELITE SOS,W N WAB,W N BADJ EM RANK,W N BADJ O RANK,W N BADJ D RANK,W N BARTHAG RANK,W N EFG% RANK,W N EFGD% RANK,W N FTR RANK,W N FTRD RANK,W N TOV% RANK,W N TOV%D RANK,W N OREB% RANK,W N DREB% RANK,W N RAW T RANK,W N 2PT% RANK,W N 2PT%D RANK,W N 3PT% RANK,W N 3PT%D RANK,W N BLK% RANK,W N BLKED% RANK,W N AST% RANK,W N OP AST% RANK,W N 2PTR RANK,W N 3PTR RANK,W N 2PTRD RANK,W N 3PTRD RANK,W N BADJT RANK,W N FT% RANK,W N OP FT% RANK,W N PPPO RANK,W N PPPD RANK,W N ELITE SOS RANK,W AN BADJ EM,W AN BADJ O,W AN BADJ D,W AN BARTHAG,W AN WIN%,W AN EFG%,W AN EFG%D,W AN FTR,W AN FTRD,W AN TOV%,W AN TOV%D,W AN OREB%,W AN DREB%,W AN RAW T,W AN 2PT%,W AN 2PT%D,W AN 3PT%,W AN 3PT%D,W AN BLK%,W AN BLKED%,W AN AST%,W AN OP AST%,W AN 2PTR,W AN 3PTR,W AN 2PTRD,W AN 3PTRD,W AN BADJ T,W AN FT%,W AN OP FT%,W AN PPPO,W AN PPPD,W AN ELITE SOS,W AN WAB,W AN BADJ EM RANK,W AN BADJ O RANK,W AN BADJ D RANK,W AN BARTHAG RANK,W AN EFG% RANK,W AN EFGD% RANK,W AN FTR RANK,W AN FTRD RANK,W AN TOV% RANK,W AN TOV%D RANK,W AN OREB% RANK,W AN DREB% RANK,W AN RA

In [12]:
# Alter format of diff to have Team Vs Opposing Team  
####################################################

# Select all winning team rows  
win1 = ordered_differentials.loc[ordered_differentials['OUTCOME'] == 1]
win1 = win1.reset_index(drop = True)    

# Select all losing team rows   
loss1 = ordered_differentials.loc[ordered_differentials['OUTCOME'] == 0]  
# Add "OPP" prefix to the opposing team 
loss1 = loss1.add_prefix('OPP ')
loss1 = loss1.reset_index(drop = True)   

# Repeat the same process as above for the opposite outcomes   
win2 = ordered_differentials.loc[ordered_differentials['OUTCOME'] == 1]
win2 = win2.add_prefix('OPP ')
win2 = win2.reset_index(drop = True)    

loss2 = ordered_differentials.loc[ordered_differentials['OUTCOME'] == 0]  
loss2 = loss2.reset_index(drop = True)   

temp1 = pd.concat([win1, loss1], axis = 1)
temp2 = pd.concat([loss2, win2], axis = 1)

# Create secondary matchups dataframe     
complete_differentials = pd.concat([temp1, temp2], axis = 0)
complete_differentials = complete_differentials.sort_index().reset_index(drop = True)
complete_differentials.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,CONF,CONF ID,QUAD NO,QUAD ID,TEAM ID,BID TYPE,OUTCOME,SCORE,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS 

In [13]:
# Display multiple subplots in one row  
def multiple_subplots(figures, titles, h) : 
    fig = make_subplots(rows = 1, cols = len(figures), subplot_titles = titles)
    
    for i, figure in enumerate(figures) :
        for trace in range(len(figure['data'])) :
            fig.append_trace(figure['data'][trace], row = 1, col = i + 1)
    
    fig.update_layout(height = h, template = 'plotly_dark')  
    return fig  

# Order dataframe by a column  
def order_df(df_input, order_by, order) :
    df_output = pd.DataFrame()

    for var in order :
        df_append = df_input[df_input[order_by] == var].copy()
        df_output = pd.concat([df_output, df_append])

    return df_output

# Change Round values to values between 0 - 7  
def change_rounds_num(df, rounds_num) : 
    for i, r in enumerate(rounds_num) : 
        df.loc[df['ROUND'] == r, 'ROUND'] = i    

# Change Round values to strings  
def change_round_str(df, rounds_num, round_str) : 
    for i, r in enumerate(rounds_num) : 
        df.loc[df['ROUND'] == r, 'ROUND'] = round_str[i]      

In [14]:
# dsadasd 

<h1 id = "MLM">
<div style = "padding:20px;color:#ffffff;margin:0;font-size:150%;text-align:center;display:fill;border-radius:5px;background-color:#0974ba;overflow:hidden;font-weight:800">
      Machine Learning Model
</div>
</h1>

In [15]:
cd = complete_differentials.copy()
cd = cd.drop(['SCORE', 'OPP SCORE'], axis = 1)
cd

YEAR  BY YEAR NO  BY ROUND NO  TEAM NO            TEAM  SEED  ROUND  \
0     2024.0      2036.0          0.0   1067.0     Connecticut   1.0    1.0   
1     2024.0      2035.0          0.0   1026.0         Stetson  16.0   64.0   
2     2024.0      2034.0          0.0   1038.0  North Carolina   1.0    1.0   
3     2024.0      2033.0          0.0   1055.0          Howard  16.0   64.0   
4     2024.0      2032.0          0.0   1038.0  North Carolina   1.0    1.0   
...      ...         ...          ...      ...             ...   ...    ...   
2031  2010.0       253.0          5.0    188.0            Duke   1.0    1.0   
2032  2009.0       128.0          4.0    101.0    Michigan St.   2.0    2.0   
2033  2009.0       127.0          3.0     95.0  North Carolina   1.0    1.0   
2034  2008.0         2.0          2.0     43.0          Kansas   1.0    1.0   
2035  2008.0         1.0          1.0     37.0         Memphis   1.0    2.0   

      CURRENT ROUND  CONF  CONF ID  QUAD NO  QUAD ID  TEAM ID  BID TYPE  \
0              64.0    BE      8.0     64.0      4.0     40.0      Auto   
1              64.0  ASun      5.0     64.0      4.0    250.0      Auto   
2              64.0   ACC      2.0     63.0      3.0    135.0  At-Large   
3              64.0     0      0.0      0.0      0.0      0.0         0   
4              64.0   ACC      2.0     63.0      3.0    135.0  At-Large   
...             ...   ...      ...      ...      ...      ...       ...   
2031            2.0   ACC      2.0      9.0      1.0     50.0      Auto   
2032            2.0   B10      6.0      8.0      4.0    115.0  At-Large   
2033            2.0   ACC      2.0      5.0      1.0    135.0  At-Large   
2034            2.0   B12      7.0      3.0      3.0     86.0      Auto   
2035            2.0  CUSA     13.0      2.0      2.0    111.0      Auto   

     OUTCOME   K TEMPO  K TEMPO RANK    KADJ T  KADJ T RANK     K OFF  \
0          1 -0.020059      0.061111 -0.088935     0.200000  0.354960   
1          0  0.020059     -0.061111  0.088935    -0.200000 -0.354960   
2          1  0.000000      0.000000  0.000000     0.000000  0.000000   
3          0  0.000000      0.000000  0.000000     0.000000  0.000000   
4          1  0.448697     -0.888889  0.478121    -0.888889  0.502927   
...      ...       ...           ...       ...          ...       ...   
2031       1  0.170291     -0.366667  0.089476    -0.194444  0.183230   
2032       0 -0.471259      0.438889 -0.337727     0.286111 -0.290239   
2033       1  0.471259     -0.438889  0.337727    -0.286111  0.290239   
2034       1 -0.082118      0.130556 -0.044548     0.069444  0.149298   
2035       0  0.082118     -0.130556  0.044548    -0.069444 -0.149298   

       KO RANK    KADJ O  KADJ O RANK     K DEF   KD RANK    KADJ D  \
0    -0.165605  0.459649    -0.297297 -0.516218 -0.922388 -0.538213   
1     0.165605 -0.459649     0.297297  0.516218  0.922388  0.538213   
2     0.000000  0.000000     0.000000  0.000000  0.000000  0.000000   
3     0.000000  0.000000     0.000000  0.000000  0.000000  0.000000   
4    -0.904459  0.618991    -0.930931 -0.053254 -0.026866 -0.342988   
...        ...       ...          ...       ...       ...       ...   
2031 -0.117834  0.240454    -0.135135 -0.053914 -0.050746 -0.080005   
2032  0.175159 -0.278753     0.102102 -0.056100 -0.077612 -0.111512   
2033 -0.175159  0.278753    -0.102102  0.056100  0.077612  0.111512   
2034 -0.054140  0.189472    -0.072072  0.124471  0.008955  0.057974   
2035  0.054140 -0.189472     0.072072 -0.124471 -0.008955 -0.057974   

      KADJ D RANK   KADJ EM  KADJ EM RANK   BADJ EM    BADJ O    BADJ D  \
0       -0.919444  0.693065     -0.697749  0.768980  0.498886 -0.579830   
1        0.919444 -0.693065      0.697749 -0.768980 -0.498886  0.579830   
2        0.000000  0.000000      0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000      0.000000  0.000000  0.000000  0.000000   
4       -0.458333  0.662964     -0.913183  0.658902  

In [16]:
selected_cols = cd.columns[~cd.columns.isin(cd.iloc[:, cd.columns.get_loc('OPP YEAR') : cd.columns.get_loc('OPP OUTCOME') + 1])]
features = cd[selected_cols]
features = features.dropna()
features = features.iloc[:, cd.columns.get_loc('K TEMPO') :]
predictors = features.columns
features.head()

,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FARTHER TWOS FG% RANK,FARTHER TWOS SHARE RANK,FARTHER TWOS FG%D RANK,FARTHER TWOS D SHARE RANK,NET RPI,RESUME,WAB

In [17]:
train = cd.loc[cd['YEAR'].le(2022)]
test = cd.loc[cd['YEAR'].eq(2023)]
train_target = train['OUTCOME'].astype(int)
test_target = test['OUTCOME'].astype(int)

In [18]:
corr_train  = train[predictors].copy()
corr_train['ROUND'] = train['ROUND']
corr_train['CURRENT ROUND'] = train['CURRENT ROUND']
corr_train = corr_train.loc[corr_train['CURRENT ROUND'].eq(64)]
cor = corr_train.corr()
cor.head()

,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FARTHER TWOS FG% RANK,FARTHER TWOS SHARE RANK,FARTHER TWOS FG%D RANK,FARTHER TWOS D SHARE RANK,NET RPI,RESUME,WAB

In [19]:
predictors = predictors[predictors.isin(['BADJ EM', 'BADJ O', 'BADJ D', 'KADJ EM', 'KADJ O', 'KADJ D',
                                         'BARTHAG', 'WAB', 'ELITE SOS', 'TALENT', 'EXP', 'POWER-PATH', 
                                         'RELATIVE RATING', 'O RATE', 'D RATE', 'OPPONENT ADJUST',  
                                         '3PT%', '3PT%D', 'TOV%', 'TOV%D', 'POOL S-RANK',
                                         'OPP BADJ EM', 'OPP BADJ O', 'OPP BADJ D', 'OPP KADJ EM', 'OPP KADJ O', 'OPP KADJ D',
                                         'OPP BARTHAG', 'OPP WAB', 'OPP ELITE SOS', 'OPP TALENT', 'OPP EXP', 'OPP POWER-PATH', 
                                         'OPP RELATIVE RATING', 'OPP O RATE', 'OPP D RATE', 'OPP OPPONENT ADJUST',  
                                         'OPP 3PT%', 'OPP 3PT%D', 'OPP TOV%', 'OPP TOV%D', 'OPP POOL S-RANK'])] 

In [20]:
threshold = 0.45    
a = abs(cor['ROUND'])
result = a[a > threshold]
result = result.sort_values(ascending = False)
print(result)

predictors = result.index
predictors = predictors.drop(['ROUND'])
print('\n', predictors)

ROUND                1.000000
OPP BADJ EM          0.625770
BADJ EM              0.625770
OPP H BADJ EM        0.621173
H BADJ EM            0.621173
                       ...   
NCAA S-RANK          0.464170
OPP A BADJ D RANK    0.453903
A BADJ D RANK        0.453903
A BADJ D             0.450929
OPP A BADJ D         0.450929
Name: ROUND, Length: 153, dtype: float64

 Index(['OPP BADJ EM', 'BADJ EM', 'OPP H BADJ EM', 'H BADJ EM', 'KADJ EM',
       'OPP KADJ EM', 'OPP BARTHAG', 'BARTHAG', 'BARTHAG RANK',
       'OPP BARTHAG RANK',
       ...
       'OPP AN BADJ D', 'AN BADJ D', 'Q2 W', 'OPP Q2 W', 'OPP NCAA S-RANK',
       'NCAA S-RANK', 'OPP A BADJ D RANK', 'A BADJ D RANK', 'A BADJ D',
       'OPP A BADJ D'],
      dtype='object', length=152)


In [21]:
space = {'n_estimators': 1000,
         'learning_rate': 0.0005,
         'max_depth': hp.quniform('max_depth', 7, 10, 1),
         'min_child_weight' : hp.quniform('min_child_weight', 0, 5, 1),
         'colsample_bytree' : hp.uniform('colsample_bytree', 0.7, 0.9),
         'gamma': hp.uniform ('gamma', 0, 0.2),
         'reg_alpha' : hp.uniform('reg_alpha', 0, 0.01),
         'reg_lambda' : hp.uniform('reg_lambda', 0.1, 0.3)}

In [22]:
def objective(space) :
    clf = XGBRegressor(n_estimators = int(space['n_estimators']), learning_rate = float(space['learning_rate']),
                       max_depth = int(space['max_depth']),  min_child_weight = int(space['min_child_weight']),
                       colsample_bytree = float(space['colsample_bytree']),
                       gamma = float(space['gamma']), reg_alpha = float(space['reg_alpha']), reg_lambda = float(space['reg_lambda']),
                       booster = 'gbtree', objective = 'binary:logistic', eval_metric = 'auc', early_stopping_rounds = 10, seed = 0)

    evaluation = [(train[predictors], train_target), (test[predictors], test_target)]
    clf.fit(train[predictors], train_target, eval_set = evaluation, verbose = False)

    pred = clf.predict(test[predictors])
    accuracy = accuracy_score(test_target, pred > 0.5)
    print ('SCORE :', accuracy)
    return {'loss': - accuracy, 'status': STATUS_OK}

In [23]:
# trials = Trials()
# best_hyperparams = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = trials)

In [24]:
# print("The best hyperparameters are : ","\n")
# print(best_hyperparams)

In [25]:
xgb = XGBRegressor(n_estimators = 1000, learning_rate = 0.0005,
                   max_depth = 8, min_child_weight = 3,
                   colsample_bytree = 0.85,
                   gamma = 0.095, reg_alpha = 0.001, reg_lambda = 0.1,
                   booster = 'gbtree', objective = 'binary:logistic', seed = 0)
xgb.fit(train[predictors], train_target)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.85, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.095, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, objective='binary:logistic', ...)

In [26]:
preds = xgb.predict(test[predictors])
preds[: 126]

array([0.3394834 , 0.659932  , 0.45673665, 0.54297054, 0.64429957,
       0.35282862, 0.41930234, 0.5641239 , 0.57160217, 0.43010747,
       0.33364403, 0.66883594, 0.54434526, 0.454598  , 0.31211013,
       0.6890363 , 0.6502278 , 0.34938964, 0.4757382 , 0.53153783,
       0.57684296, 0.4270914 , 0.66853577, 0.3308388 , 0.4889809 ,
       0.51105446, 0.3797449 , 0.61553085, 0.4272647 , 0.57778895,
       0.6585553 , 0.33641562, 0.31105104, 0.6894615 , 0.53646034,
       0.46360976, 0.578996  , 0.42394018, 0.43803722, 0.56392705,
       0.5319012 , 0.4703262 , 0.3490334 , 0.6458676 , 0.4416701 ,
       0.5564158 , 0.64082354, 0.3557439 , 0.61590785, 0.38057297,
       0.41309962, 0.5842203 , 0.60166585, 0.3948901 , 0.68217385,
       0.3171184 , 0.45622674, 0.54320735, 0.6963225 , 0.30323383,
       0.4862774 , 0.5138075 , 0.34809777, 0.6531289 , 0.60911727,
       0.3878907 , 0.6544204 , 0.34350643, 0.43091384, 0.57331586,
       0.3337776 , 0.66363   , 0.68622327, 0.31350663, 0.59323

In [27]:
accuracy = accuracy_score(test_target, preds > 0.5)
print(accuracy)

0.6984126984126984


In [28]:
def get_leverage(winner, team, game, game_pred, seed_diff) : 
    #              1     2      3     4    5    6     7      8      9     10    11    12    13   14   15
    win_prob = [0.475, 0.45, 0.425, 0.4, 0.4, 0.4, 0.375, 0.375, 0.35, 0.35, 0.35, 0.35, 0.5, 0.5, 0.5]
    
    # If the seed differential is a specified number 
    if (abs(game['SEED'][team] - game['SEED'][team + 1] == seed_diff)) :
        # If the first team's seed is greater than the second team's seed 
        if game['SEED'][team] > game['SEED'][team + 1] :
            # The first team is the winner if they surpass the altered win probability threshold 
            if game_pred[0] > win_prob[seed_diff - 1] : winner = game.loc[team]
        else :
            # The second team is the winner if they surpass the altered win probability threshold 
            if game_pred[1] > win_prob[seed_diff - 1] : winner = game.loc[team + 1]

    return winner 

In [29]:
def simulate_tourney(round_df, mode) :
    removed_cols = ['YEAR', 'BY YEAR NO', 'BY ROUND NO', 'TEAM NO', 'TEAM', 'SEED', 'ROUND', 'CURRENT ROUND', 'CONF', 'CONF ID', 'QUAD NO', 'QUAD ID', 'TEAM ID', 'BID TYPE', 'OUTCOME', 'TYPE']
    selected_cols = round_df.columns[~round_df.columns.isin(removed_cols)]
    game_count = 32
    curr_df = round_df.copy()

    for i in range(6) :
        team = 0
        next_round = pd.DataFrame()

        # Get the team's stats (not including the opposition team)
        if i != 0 : round_df = round_df.iloc[:, : round_df.columns.get_loc('OPP YEAR')]
        # Replace all N/A values with 0 values
        round_df = round_df.fillna(0)
        # Set the TYPE value to 0 for even indexes and 1 for odd indexes
        round_df['TYPE'] = np.where(round_df.index % 2 == 0, 0, 1)

        # Get the difference of every stat for every two rows
        odds = round_df[selected_cols].diff()
        # Select all odd indexes
        odds_df = odds.iloc[1::2]

        # Get the difference of every stat for every two rows
        evens = - round_df[selected_cols].diff()
        # Shift the values up one row
        evens = evens[selected_cols].shift(- 1)
        # Select all even indexes
        evens_df = evens.iloc[::2]

        temp = round_df[removed_cols]

        # Combine the temp, odds_df, and evens_df to make one dataframe consisting of all tournament matchup differentials
        round_df = pd.concat([temp, evens_df], axis = 1)
        round_df = pd.concat([round_df, odds_df], axis = 0)
        round_df = round_df.groupby(level = 0).sum()
        round_df = round_df.sort_index(ascending = True)

        # Select the teams listed first in the matchup
        win1 = round_df.loc[round_df['TYPE'] == 1]
        win1 = win1.reset_index(drop = True)

        # Select the teams listed second in the matchup
        loss1 = round_df.loc[round_df['TYPE'] == 0]
        # Give every column from the second team the "OPP" prefix to represent the opposing team
        loss1 = loss1.add_prefix('OPP ')
        loss1 = loss1.reset_index(drop = True)

        win2 = round_df.loc[round_df['TYPE'] == 1]
        win2 = win2.add_prefix('OPP ')
        win2 = win2.reset_index(drop = True)

        loss2 = round_df.loc[round_df['TYPE'] == 0]
        loss2 = loss2.reset_index(drop = True)

        # Combine dataframes to create all tournament matchups
        temp1 = pd.concat([win1, loss1], axis = 1)
        temp2 = pd.concat([loss2, win2], axis = 1)

        round_df = pd.concat([temp1, temp2], axis = 0)
        round_df = round_df.sort_index().reset_index(drop = True)

        for i in range(int(game_count)) :
            # Get the specific matchup
            game = round_df.loc[team : team + 1]

            # Predict the matchup's win probability
            game_pred = xgb.predict(game[predictors])

            # Select the winner of the matchup
            if game_pred[0] > game_pred[1] : winner = game.loc[team]
            else : winner = game.loc[team + 1]

            if game_pred[0] > game_pred[1] : winner = game.loc[team]
            else : winner = game.loc[team + 1]

            # Alternate method of selecting the winner of the matchup
            if mode == 1 :
                seed_diff = abs(game.loc[team, 'SEED'] - game.loc[team + 1, 'SEED']) 
                if seed_diff != 0 : winner = get_leverage(winner, team, game, game_pred, int(seed_diff))
                else : 
                    if game_pred[0] > game_pred[1] : winner = game.loc[team]
                    else : winner = game.loc[team + 1]

                    if game_pred[0] > game_pred[1] : winner = game.loc[team]
                    else : winner = game.loc[team + 1]

            winner = winner.to_frame().T
            winner = curr_df[curr_df['TEAM'].isin(winner['TEAM'])]
            next_round = pd.concat([next_round, winner], axis = 0)
            team += 2

        next_round = next_round.reset_index(drop = True)

        odds = next_round.iloc[1::2]
        evens = next_round.iloc[::2]

        if game_count != 1 :
            odds = odds.set_index(np.arange(0, game_count, 2))
            evens = evens.set_index(np.arange(1, game_count, 2))

        # Create new dataframe to display the outcomes of the current round
        opp = pd.concat([evens, odds], axis = 0)
        opp = opp.add_prefix('OPP ')
        opp = opp.sort_index(ascending = True)
        next_round = pd.concat([next_round, opp], axis = 1)
        next_round = next_round.reset_index(drop = True)
        next_round['CURRENT ROUND'] = game_count
        round_df = next_round.copy()
        # Reduce game count by half as the subsequent round has half the number of games as the current round
        game_count /= 2

        # Display the outcomes of the current round
        display(round_df[['YEAR', 'TEAM', 'SEED', 'CURRENT ROUND']])

In [30]:
curr_matchups = ordered_tournament_matchups.copy()
# Get the current tournament matchups
curr_matchups = curr_matchups.loc[curr_matchups['YEAR'].eq(2024) & curr_matchups['CURRENT ROUND'].eq(64)]

# Get the matchups in a specific order in order for the simulation to be able to read the dataframe properly
curr_tourney = ts_df.copy()
curr_tourney = curr_tourney.drop(['YEAR', 'BY ROUND NO', 'TEAM NO', 'SEED', 'ROUND', 'CURRENT ROUND'], axis = 1)
curr_tourney = pd.merge(curr_tourney, curr_matchups, on = 'BY YEAR NO', how = 'left')
curr_tourney = curr_tourney.drop(['TEAM_y'], axis = 1)
curr_tourney = curr_tourney.rename({'TEAM_x' : 'TEAM'}, axis = 1)
curr_tourney.head()

,BY YEAR NO,TEAM,YEAR,BY ROUND NO,TEAM NO,SEED,ROUND,CURRENT ROUND,SCORE,CONF,CONF ID,QUAD NO,QUAD ID,TEAM ID,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,H BADJ EM,H BADJ O,H BADJ D,H BARTHAG,H WIN%,H EFG%,H EFG%D,H FTR,H FTRD,H TOV%,H TOV%D,H OREB%,H DREB%,H RAW T,H 2PT%,H 2PT%D,H 3PT%,H 3PT%D,H BLK%,H BLKED%,H AST%,H OP AST%,H 2PTR,H 3PTR,H 2PTRD,H 3PTRD,H BADJ T,H FT%,H OP FT%,H PPPO,H PPPD,H ELITE SOS,H WAB,H BADJ EM RANK,H BADJ O RANK,H BADJ D RANK,H BARTHAG RANK,H EFG% RANK,H EFGD% RANK,H FTR RANK,H FTRD RANK,H TOV% RANK,H TOV%D RANK,H OREB% RANK,H DREB% RANK,H RAW T RANK,H 2PT% RANK,H 2PT%D RANK,H 3PT% RANK,H 3PT%D RANK,H BLK% RANK,H BLKED% RANK,H AST% RANK,H OP AST% RANK,H 2PTR RANK,H 3PTR RANK,H 2PTRD RANK,H 3PTRD RANK,H BADJT RANK,H FT% RANK,H OP FT% RANK,H PPPO RANK,H PPPD RANK,H ELITE SOS RANK,A BADJ EM,A BADJ O,A BADJ D,A BARTHAG,A WIN%,A EFG%,A EFG%D,A FTR,A FTRD,A TOV%,A TOV%D,A OREB%,A DREB%,A RAW T,A 2PT%,A 2PT%D,A 3PT%,A 3PT%D,A BLK%,A BLKED%,A AST%,A OP AST%,A 2PTR,A 3PTR,A 2PTRD,A 3PTRD,A BADJ T,A FT%,A OP FT%,A PPPO,A PPPD,A ELITE SOS,A WAB,A BADJ EM RANK,A BADJ O RANK,A BADJ D RANK,A BARTHAG RANK,A EFG% RANK,A EFGD% RANK,A FTR RANK,A FTRD RANK,A TOV% RANK,A TOV%D RANK,A OREB% RANK,A DREB% RANK,A RAW T RANK,A 2PT% RANK,A 2PT%D RANK,A 3PT% RANK,A 3PT%D RANK,A BLK% RANK,A BLKED% RANK,A AST% RANK,A OP AST% RANK,A 2PTR RANK,A 3PTR RANK,A 2PTRD RANK,A 3PTRD RANK,A BADJT RANK,A FT% RANK,A OP FT% RANK,A PPPO RANK,A PPPD RANK,A ELITE SOS RANK,N BADJ EM,N BADJ O,N BADJ D,N BARTHAG,N WIN%,N EFG%,N EFG%D,N FTR,N FTRD,N TOV%,N TOV%D,N OREB%,N DREB%,N RAW T,N 2PT%,N 2PT%D,N 3PT%,N 3PT%D,N BLK%,N BLKED%,N AST%,N OP AST%,N 2PTR,N 3PTR,N 2PTRD,N 3PTRD,N BADJ T,N FT%,N OP FT%,N PPPO,N PPPD,N ELITE SOS,N WAB,N BADJ EM RANK,N BADJ O RANK,N BADJ D RANK,N BARTHAG RANK,N EFG% RANK,N EFGD% RANK,N FTR RANK,N FTRD RANK,N TOV% RANK,N TOV%D RANK,N OREB% RANK,N DREB% RANK,N RAW T RANK,N 2PT% RANK,N 2PT%D RANK,N 3PT% RANK,N 3PT%D RANK,N BLK% RANK,N BLKED% RANK,N AST% RANK,N OP AST% RANK,N 2PTR RANK,N 3PTR RANK,N 2PTRD RANK,N 3PTRD RANK,N BADJT RANK,N FT% RANK,N OP FT% RANK,N PPPO RANK,N PPPD RANK,N ELITE SOS RANK,AN BADJ EM,AN BADJ O,AN BADJ D,AN BARTHAG,AN WIN%,AN EFG%,AN EFG%D,AN FTR,AN FTRD,AN TOV%,AN TOV%D,AN OREB%,AN DREB%,AN RAW T,AN 2PT%,AN 2PT%D,AN 3PT%,AN 3PT%D,AN BLK%,AN BLKED%,AN AST%,AN OP AST%,AN 2PTR,AN 3PTR,AN 2PTRD,AN 3PTRD,AN BADJ T,AN FT%,AN OP FT%,AN PPPO,AN PPPD,AN ELITE SOS,AN WAB,AN BADJ EM RANK,AN BADJ O RANK,AN BADJ D RANK,AN BARTHAG RANK,AN EFG% RANK,AN EFGD% RANK,AN FTR RANK,AN FTRD RANK,AN TOV% RANK,AN TOV%D RANK,AN OREB% RANK,AN DREB% RANK,AN RAW T RANK,AN 2PT% RANK,AN 2PT%D RANK,AN 3PT% RANK,AN 3PT%D RANK,AN BLK% RANK,AN BLKED% RANK,AN AST% RANK,AN OP AST% RANK,AN 2PTR RANK,AN 3PTR RANK,AN 2PTRD RANK,AN 3PTRD RANK,AN BADJT RANK,AN FT% RANK,AN OP FT% RANK,AN PPPO RANK,AN PPPD RANK,AN ELITE SOS RANK,POWER,PATH,DRAW,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,AP VOTES,AP RANK,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,CLOSE TWOS SHARE,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,DUNKS FG% RANK,DUNKS SHARE RANK,DUNKS FG%D RANK,DUNKS D SHARE RANK,CLOSE TWOS FG% RANK,CLOSE TWOS SHARE RANK,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FART

In [31]:
# Run the tournament simulation
simulate_tourney(curr_tourney, 0)

,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,32
1,2024,Florida Atlantic,8,32
2,2024,San Diego St.,5,32
3,2024,Auburn,4,32
4,2024,BYU,6,32
5,2024,Illinois,3,32
6,2024,Washington St.,7,32
7,2024,Iowa St.,2,32
8,2024,North Carolina,1,32
9,2024,Michigan St.,9,32


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,16.0
1,2024,Auburn,4,16.0
2,2024,Illinois,3,16.0
3,2024,Iowa St.,2,16.0
4,2024,North Carolina,1,16.0
5,2024,Saint Mary's,5,16.0
6,2024,Baylor,3,16.0
7,2024,Arizona,2,16.0
8,2024,Houston,1,16.0
9,2024,Duke,4,16.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,8.0
1,2024,Illinois,3,8.0
2,2024,North Carolina,1,8.0
3,2024,Arizona,2,8.0
4,2024,Houston,1,8.0
5,2024,Marquette,2,8.0
6,2024,Purdue,1,8.0
7,2024,Tennessee,2,8.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,4.0
1,2024,Arizona,2,4.0
2,2024,Houston,1,4.0
3,2024,Purdue,1,4.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,2.0
1,2024,Purdue,1,2.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Purdue,1,1.0


In [32]:
# Run the upset tournament simulation
simulate_tourney(curr_tourney, 1)

,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,32
1,2024,Florida Atlantic,8,32
2,2024,UAB,12,32
3,2024,Auburn,4,32
4,2024,BYU,6,32
5,2024,Illinois,3,32
6,2024,Drake,10,32
7,2024,Iowa St.,2,32
8,2024,North Carolina,1,32
9,2024,Michigan St.,9,32


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Connecticut,1,16.0
1,2024,Auburn,4,16.0
2,2024,BYU,6,16.0
3,2024,Iowa St.,2,16.0
4,2024,Michigan St.,9,16.0
5,2024,Alabama,4,16.0
6,2024,Baylor,3,16.0
7,2024,Arizona,2,16.0
8,2024,Houston,1,16.0
9,2024,Duke,4,16.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Auburn,4,8.0
1,2024,Iowa St.,2,8.0
2,2024,Michigan St.,9,8.0
3,2024,Arizona,2,8.0
4,2024,Houston,1,8.0
5,2024,Florida,7,8.0
6,2024,Purdue,1,8.0
7,2024,Tennessee,2,8.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Auburn,4,4.0
1,2024,Arizona,2,4.0
2,2024,Houston,1,4.0
3,2024,Purdue,1,4.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Arizona,2,2.0
1,2024,Purdue,1,2.0


,YEAR,TEAM,SEED,CURRENT ROUND
0,2024,Purdue,1,1.0
